In [3]:
import sys
sys.path.insert(0, "..")

from tqdm import tqdm
import torch
from my_datasets import *

In [4]:
n, r, ap, bp, cl = 8, 8, 0.2, 0.2, 4

In [20]:
def old_compose(rules1, rules2):
    all_as, all_bs = [], []
    for r1 in rules1:
        a, b = r1.chunk(2)
        z, _ = step_rules(rules1[None,...], a.view(1,-1))
        new_b, _ = step_rules(rules2[None,...], z.view(1,-1))
        all_as.append(a)
        all_bs.append(new_b)
    all_as = torch.stack(all_as)
    all_bs = torch.cat(all_bs, dim=0)
    return torch.cat([all_as, all_bs], dim=1).long()

def compose(rules1, rules2):
    all_as, all_bs = [], []
    for r1 in rules1:
        a, b = r1.chunk(2)
        z, _ = step_rules(rules1[None,...], a.view(1,-1))
        # new_b, _ = step_rules(rules2[None,...], z.view(1,-1))
        new_b = z

        # all_as.append(a)
        
        y, _ = step_rules(rules1[None,...], torch.zeros(1,n))
        all_as.append((a - y.squeeze()).clamp(0,1))
        all_bs.append(new_b)
    all_as = torch.stack(all_as)
    all_bs = torch.cat(all_bs, dim=0)
    return torch.cat([all_as, all_bs], dim=1).long()

# def compose(rules1, rules2):
#     all_as, all_bs = [], []
#     for r1 in rules1:
#         a, b = r1.chunk(2)
#         z, _ = step_rules(rules1[None,...], a.view(1,-1))
#         new_b, _ = step_rules(rules2[None,...], z.view(1,-1))
#         all_as.append(a)
#         all_bs.append(new_b)

#     for r1 in rules1:
#         a1, b1 = r1.chunk(2)
#         for r2 in rules2:
#             a2, b2 = r2.chunk(2)
#             all_as.append((a1 + (a2 - b1).clamp(0,1)).clamp(0,1))
#             all_bs.append((b1 + b2).clamp(0,1).view(1,-1))
            
#             all_as.append((a2 + (a1 - b2).clamp(0,1)).clamp(0,1))
#             all_bs.append((b1 + b2).clamp(0,1).view(1,-1))
#     all_as = torch.stack(all_as)
#     all_bs = torch.cat(all_bs, dim=0)
#     return torch.cat([all_as, all_bs], dim=1).long()


In [21]:
torch.manual_seed(1234)
pbar = tqdm(range(100))
for i in pbar:
    dictA = random_rules_with_chain(
        num_rules = r,
        num_vars = n,
        ante_prob = ap,
        conseq_prob = bp,
        chain_len = cl,
        return_dict = True)
    rulesA = dictA["rules"]

    dictB = random_rules_with_chain(
        num_rules = r,
        num_vars = n, 
        ante_prob = ap,
        conseq_prob = bp,
        chain_len = cl,
        return_dict = True)
    rulesB = dictB["rules"]

    rulesB[:,0:n] = rulesA[:,0:n]

    s0 = torch.zeros(1,n)
    # s0 = torch.randint(0,2,(1,n))

    rulesAA = compose(rulesA, rulesA)
    rulesAAA1 = compose(rulesA, rulesAA)
    rulesAAA2 = compose(rulesAA, rulesA)
    rulesAAAA = compose(rulesAA, rulesAA)
    rulesAB = compose(rulesA, rulesB)

    sa, _ = step_rules(rulesA[None,...], s0)
    saa, _ = step_rules(rulesA[None,...], sa)
    sab, _ = step_rules(rulesB[None,...], sa)
    saaa = kstep_rules(rulesA[None,...], s0, num_steps=3)
    saaaa = kstep_rules(rulesA[None,...], s0, num_steps=4)
                      
    scaa, _ = step_rules(rulesAA[None,...], s0)
    scab, _ = step_rules(rulesAB[None,...], s0)

    scaaa1, _ = step_rules(rulesAAA1[None,...], s0)
    scaaa2, _ = step_rules(rulesAAA2[None,...], s0)
    scaaaa, _ = step_rules(rulesAAAA[None,...], s0)

    aa_diff = (saa - scaa).abs().sum()
    ab_diff = (sab - scab).abs().sum()

    aaa1_diff = (saaa - scaaa1).abs().sum()
    aaa2_diff = (saaa - scaaa2).abs().sum()
    aaaa_diff = (saaaa - scaaaa).abs().sum()

    if aa_diff > 0 or ab_diff > 0 or aaa1_diff > 0 or aaa2_diff > 0 or aaaa_diff > 0:
        break

  0%|          | 0/100 [00:00<?, ?it/s]


In [22]:
aa_diff

tensor(0)

In [23]:
saa, scaa

(tensor([[0, 1, 0, 0, 1, 1, 1, 1, 0, 1]]),
 tensor([[0, 1, 0, 0, 1, 1, 1, 1, 0, 1]]))

In [24]:
qed = prove_theorem(rulesA[None,...], torch.ones(1,n))
qed

{'rules': tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
          [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
          [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
          [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
          [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]]]),
 'theorem': tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
 'qed': tensor([0]),
 'states': tensor([[[0, 1, 0, 0, 0, 1, 1, 0, 0, 1],
          [0, 1, 0, 0, 1, 1, 1, 1, 0, 1],
          [1, 1, 0, 0, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
     

In [25]:
n, r, ap, bp, cl = 10, 10, 0.2, 0.2, 4

In [26]:
torch.manual_seed(1234)
pbar = tqdm(range(10000))
for i in pbar:
    # dictA = random_rules_with_chain(
    #     num_rules = r,
    #     num_vars = n,
    #     ante_prob = ap,
    #     conseq_prob = bp,
    #     chain_len = cl,
    #     num_fillers = 0,
    #     return_dict = True)
    # rulesA = dictA["rules"]

    rulesA = (torch.rand(r,2*n) < 0.4).long()

    # s0 = rulesA[0:1,0:n]
    s0 = torch.zeros(1,n)
    # s0 = torch.randint(0,2,(1,n))
    rulesComp1 = compose(compose(rulesA, rulesA), compose(rulesA, rulesA))
    rulesComp2 = compose(compose(compose(rulesA, rulesA), rulesA), rulesA)
    
    sc1, _ = step_rules(rulesComp1[None,...], s0)
    sc2, _ = step_rules(rulesComp2[None,...], s0)
    sa4 = kstep_rules(rulesA[None,...], s0, num_steps=4)
    
    sc_diff = (sc1 - sc2).abs().sum()
    # if sc_diff > 0:
    #     break

    sa_diff = (sc1 - sa4).abs().sum()
    if sa_diff > 0:
        break

    # rc_diff = (rulesComp1 - rulesComp2).abs().sum()
    # if rc_diff > 0:
        # break

  2%|▏         | 248/10000 [00:01<00:58, 166.68it/s]


In [ ]:
rulesA

In [ ]:
compose(rulesA, rulesA)

In [ ]:
step_rules(rulesA[None,...], torch.tensor([[0,0,0,1,1]]))

In [ ]:
rulesComp1

In [ ]:
sc1, sa4

In [ ]:
qed = prove_theorem(rulesA[None,...], torch.ones(1,n))
qed

In [ ]:
# rulesT = torch.tensor([
#     [1,0,0,0,0, 0,0,0,1,0],
#     [0,1,0,0,0, 1,0,1,0,0],
#     [0,0,0,0,0, 0,1,0,0,0],
#     [0,0,1,1,0, 0,0,0,0,1]
# ])

rulesT = torch.tensor([
    [0,0,0,0,0, 1,0,0,0,0],
    [1,0,0,0,0, 0,1,1,0,0],
    [0,1,0,0,0, 0,0,0,1,0],
    [0,0,1,1,0, 0,0,0,0,1]
])

In [ ]:
rulesTC = compose(compose(rulesT, rulesT), compose(rulesT, rulesT))
stc, _ = step_rules(rulesTC[None,...], torch.zeros(1,5))
st4 = kstep_rules(rulesTC[None,...], torch.zeros(1,5), num_steps=4)

In [ ]:
stc, st4

In [ ]:
stc